In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True' 

In [ ]:
# 하이퍼 파라미터 튜닝

from ultralytics import YOLO
model = YOLO("yolo11l-obb.pt")
model.tune(
    data = "data_com.yaml", 
    epochs = 10, 
    imgsz = 768,
    batch=2,
    # cfg = "runs/obb/tune4/best_hyperparameters.yaml",
    iterations=100, 
    optimizer="AdamW",
    name ="tunning_768_label",
    flipud = 0.5, 
    scale = 0.88, 
    bgr = 0.5,
    # label_smoothing = 0.3,
    )

Tuner: Initialized Tuner instance with 'tune_dir=runs\obb\tune15'
Tuner:  Learn about tuning at https://docs.ultralytics.com/guides/hyperparameter-tuning
Tuner: Starting iteration 1/100 with hyperparameters: {'lr0': 0.01, 'lrf': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'warmup_epochs': 3.0, 'warmup_momentum': 0.8, 'box': 7.5, 'cls': 0.5, 'dfl': 1.5, 'hsv_h': 0.015, 'hsv_s': 0.7, 'hsv_v': 0.4, 'degrees': 0.0, 'translate': 0.1, 'scale': 0.88, 'shear': 0.0, 'perspective': 0.0, 'flipud': 0.5, 'fliplr': 0.5, 'bgr': 0.5, 'mosaic': 1.0, 'mixup': 0.0, 'copy_paste': 0.0}


In [1]:
# 직접 YOLO 클래스를 임포트
from ultralytics import YOLO

base = "yolo11l-obb.pt"
resume ="runs/obb/train13/weights/best.pt"
# YOLO 모델 초기화
model = YOLO(base) 
results = model.train(
    data="data_com.yaml",
    name = "l_s1024_bgr05_scale084_",
    epochs=500,
    batch=0.8,
    imgsz=1024,
    patience=15,
    device="0",
    optimizer = "AdamW",
    lr0= 0.00088,
    lrf= 0.01134,
    momentum= 0.82858,
    weight_decay= 0.00028,
    warmup_epochs= 2.33952,
    warmup_momentum= 0.58988,
    box= 5.06927,
    cls= 0.56241,
    dfl= 1.36767,
    hsv_h= 0.01707,
    hsv_s= 0.68688,
    hsv_v= 0.28102,
    degrees= 0.0,
    translate= 0.11263,
    shear= 0.0,
    perspective= 0.0,
    flipud= 0.45936,
    fliplr= 0.46892,
    bgr= 0.5,
    mosaic= 0.828,
    mixup= 0.0,
    copy_paste= 0.0,
    scale = 0.92,
    
    dropout = 0.35,
    label_smoothing = 0.3,
    plots = True,
    )

New https://pypi.org/project/ultralytics/8.3.15 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
engine\trainer: task=obb, mode=train, model=yolo11l-obb.pt, data=data_com.yaml, epochs=500, time=None, patience=15, batch=0.8, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=l_s1024_bgr05_scale084_2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.35, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, s

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: woow070840 (waooang). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLO11n...
AMP: checks passed 
AutoBatch: Computing optimal batch size for imgsz=1024 at 80.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4070 Ti) 11.99G total, 0.26G reserved, 0.24G allocated, 11.50G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
    26160857       232.9         1.856         59.15           nan      (1, 3, 1024, 1024)                    list
    26160857       465.8         4.941         69.81           nan      (2, 3, 1024, 1024)                    list
    26160857       931.6         9.578           116           nan      (4, 3, 1024, 1024)                    list
    26160857        1863        18.862          7896           nan      (8, 3, 1024, 1024)                    list
CUDA out of memory. Tried to allocate 128.00 MiB. GPU 0 has a total capacity of 11.99 GiB of which 0

train: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\train.cache... 876 images, 140 backgrounds, 0 corrupt: 100%|██████████| 876/876 [00:00<?, ?it/s]
val: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]


Plotting labels to runs\obb\l_s1024_bgr05_scale084_2\labels.jpg... 
optimizer: AdamW(lr=0.00088, momentum=0.82858) with parameter groups 173 weight(decay=0.0), 183 weight(decay=0.000275625), 182 bias(decay=0.0)
Image sizes 1024 train, 1024 val
Using 8 dataloader workers
Logging results to runs\obb\l_s1024_bgr05_scale084_2
Starting training for 500 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         92       4910      0.738      0.688      0.711      0.513
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 72, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



87 epochs completed in 1.228 hours.
Optimizer stripped from runs\obb\l_s1024_bgr05_scale084_2\weights\last.pt, 53.1MB
Optimizer stripped from runs\obb\l_s1024_bgr05_scale084_2\weights\best.pt, 53.1MB

Validating runs\obb\l_s1024_bgr05_scale084_2\weights\best.pt...
Ultralytics 8.3.13  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11l-obb summary (fused): 483 layers, 26,129,305 parameters, 0 gradients, 90.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  7.31it/s]


                   all         92       4910      0.788        0.7      0.741      0.539
               ship(s)         79       4334      0.899      0.661      0.743      0.524
               ship(L)         61        576      0.677      0.738      0.738      0.554
Speed: 0.7ms preprocess, 15.2ms inference, 0.0ms loss, 7.4ms postprocess per image
Results saved to runs\obb\l_s1024_bgr05_scale084_2


lr/pg0,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▁▆█████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆
lr/pg2,▂████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▁▁▁▁
metrics/mAP50(B),▂▂▄▄▄▄▄▄▃▅▆▅▅▆▆▇▇▆▇▇▇▇▇▇▇█▇██▇██▇▇▁▄█▇▇█
metrics/mAP50-95(B),▃▄▃▃▄▃▁▄▃▃▄▅▅▆▆▇▆▆▇▆▆▆▇▇▇▇▇██▆██▇▇▃█▅▇██
metrics/precision(B),▃▁▂▄▅▄▄▄▁▅▅▅▅▅▆▇▄▆▆█▆▆▇▇▇▄█▇▅█▇▆▇▇▇▆▇▃█▇
metrics/recall(B),▁▄▃▃▂▃▂▃▃▂▅▄▄▄▅▃▅▅▅▆▅▅▆▇▆▆▆▅▆█▇▇▅▆▇▇▆▆▅▆
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,██▇▇▇▆▇▆▆▅▆▅▄▆▅▃▅▃▃▄▃▃▃▄▃▁▂▃▃▃▁▂▂▂▂▃▁▂▁▁


In [6]:
model = YOLO("runs\obb\combine_1024\weights/best.pt")

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

<>:1: SyntaxWarning: invalid escape sequence '\o'
<>:1: SyntaxWarning: invalid escape sequence '\o'
C:\Users\Seungwoo\AppData\Local\Temp\ipykernel_35504\4051414092.py:1: SyntaxWarning: invalid escape sequence '\o'
  model = YOLO("runs\obb\combine_1024\weights/best.pt")


Ultralytics 8.3.9  Python-3.12.7 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
YOLO11x-obb summary (fused): 483 layers, 58,737,913 parameters, 0 gradients, 202.7 GFLOPs


val: Scanning E:\Workspace\SatellieteIMGDetection\datasets\labels\val.cache... 92 images, 0 backgrounds, 0 corrupt: 100%|██████████| 92/92 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:14<00:00,  2.34s/it]


                   all         92       4910      0.877      0.688      0.782      0.591
               ship(s)         79       4334      0.913      0.699      0.781       0.58
               ship(L)         61        576      0.841      0.677      0.782      0.602
Speed: 1.1ms preprocess, 106.3ms inference, 0.0ms loss, 8.8ms postprocess per image
Results saved to runs\obb\val


array([    0.58025,     0.60218])

In [ ]:
# # YOLO 모델의 Detect 모듈을 제외한 백본만 추출
# backbone = nn.Sequential(*list(yolo.model.model.children())[:-1])
# backbone

In [ ]:
# head = nn.Sequential(list(yolo.model.model.children())[-1])
# head

In [1]:
# import torch
# import torch.nn as nn
# from ultralytics import YOLO

# # YOLO 모델 불러오기 (YOLOv5 예시)
# class YOLOv5WithCustomHead(nn.Module):
#     def __init__(self, yolo):
#         super(YOLOv5WithCustomHead, self).__init__()
#         self.yolo = yolo.model  # YOLO 모델의 모든 레이어
        
#         # YOLO의 중간층에서 사용할 레이어 인덱스를 확인해야 함
#         self.custom_head = nn.Sequential(
#             nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(),
#             nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling
#             nn.Flatten(),
#             nn.Linear(128, 10)  # 예시로 10개 클래스를 분류하는 헤드
#         )

#     def forward(self, x):
#         outputs = []  # 여러 텐서를 저장할 리스트
        
#         # 중간 피처 맵을 추출하는 부분
#         for i, layer in enumerate(self.yolo.model):
#             x = layer(x)
#             if isinstance(x, list):  # x가 리스트인 경우
#                 x = torch.cat(x, dim=1)  # 리스트의 텐서를 결합
#             if i == 9:  # 예시로 9번째 레이어 출력 사용
#                 feature_map = x  # 중간 피처 맵 추출
#                 outputs.append(feature_map)  # 필요한 피처 맵 저장

#         # 기존 YOLO 아웃풋 (객체 탐지)
#         yolo_output = x

#         # 새로운 헤드에서 중간 피처 맵 사용
#         custom_output = self.custom_head(feature_map)
        
#         return yolo_output, custom_output

# # 기존 YOLOv5 모델 불러오기
# yolo = YOLO('yolo11m.pt')

# # 커스텀 모델 생성 (YOLOv5에 새로운 헤드 추가)
# model_with_custom_head = YOLOv5WithCustomHead(yolo)

# # 예시 입력 데이터
# input_data = torch.randn(1, 3, 640, 640)

# # 모델 실행
# yolo_output, custom_output = model_with_custom_head(input_data)

# # 결과 출력
# print("YOLO Output Shape:", yolo_output.shape)
# print("Custom Head Output Shape:", custom_output.shape)



TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


In [9]:
yolo.model

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(192, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(256, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   